Start jupyter-lab

```bash
jupyter-lab --notebook-dir=$HOME/gits/gerashegalov/rapids-shell/src/jupyter
```
or simply open in VS Code with Jupyter extension

# Conv

In [1]:
import findspark

### Versions

In [2]:
cuda_version = 'cuda11'
hadoop_version = '3'
java_version = '8'
rapids_version = '23.10.0-SNAPSHOT'
scala_version = '2.12'
spark_version = '3.3.2'

### Environment Variables

In [3]:
os.environ['JAVA_HOME'] = f"/usr/lib/jvm/java-{java_version}-openjdk-amd64"
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
os.environ['TZ'] = 'UTC'

In [4]:
home_dir = os.environ['HOME']
work_dir = f"{home_dir}/jupyter_run_dir"
m2_local_repo = f"{home_dir}/.m2/repository"
groupId = "com.nvidia"
artifactId = f"rapids-4-spark_{scala_version}"
dist_jar = f"{artifactId}-{rapids_version}-{cuda_version}.jar"
spark_home = f"{home_dir}/dist/spark-{spark_version}-bin-hadoop{hadoop_version}"
# spark_home = f"{home_dir}/gits/apache/spark"

### Find Spark 

In [5]:
findspark.init(spark_home = spark_home)
findspark.add_jars(f"{m2_local_repo}/com/nvidia/{artifactId}/{rapids_version}/{dist_jar}")

import pyspark

### Configure Spark

In [6]:
cores_per_exec = 1
jdwp = '-agentlib:jdwp=transport=dt_socket,server=y,suspend=n,address=5005'
spark_master = f"local[{cores_per_exec}]"
spark_builder = pyspark.sql.SparkSession.builder
spark_builder.config('spark.app.name', 'RAPIDS PySpark Notebook')
spark_builder.config('spark.driver.extraJavaOptions', f"-Dai.rapids.cudf.preserve-dependencies=true {jdwp}")
spark_builder.config('spark.master', spark_master)
spark_builder.config('spark.plugins', 'com.nvidia.spark.SQLPlugin')
spark_builder.config('spark.rapids.sql.explain', 'ALL')

spark = spark_builder.getOrCreate()

Listening for transport dt_socket at address: 5005
23/07/31 04:54:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/31 04:54:48 WARN RapidsPluginUtils: RAPIDS Accelerator 23.10.0-SNAPSHOT using cudf 23.10.0-SNAPSHOT.
23/07/31 04:54:48 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.
23/07/31 04:54:48 WARN RapidsPluginUtils: spark.rapids.sql.explain is set to `ALL`. Set it to 'NONE' to suppress the diagnostics logging about the query placement on the GPU.


# Test Scenario

In [7]:
df = spark.createDataFrame([('gera', 2), ('231', 10), ('0xDEADBEEF', 16),], 'a string, b integer')
df.printSchema()
df.createOrReplaceTempView("TBL")
df.show(truncate = False)

root
 |-- a: string (nullable = true)
 |-- b: integer (nullable = true)

23/07/31 04:54:59 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
  *Exec <ProjectExec> will run on GPU
    *Expression <Alias> cast(b#1 as string) AS b#7 will run on GPU
      *Expression <Cast> cast(b#1 as string) will run on GPU
    ! <RDDScanExec> cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.execution.RDDScanExec
      @Expression <AttributeReference> a#0 could run on GPU
      @Expression <AttributeReference> b#1 could run on GPU



+----------+---+
|a         |b  |
+----------+---+
|gera      |2  |
|231       |10 |
|0xDEADBEEF|16 |
+----------+---+



In [8]:
from enum import Enum
class Chip(Enum):
    CPU = 1
    GPU = 2

In [9]:
def test_case(chip):
    print(f"##### TEST {chip=}\n")
    spark.conf.set('spark.rapids.sql.enabled', chip == Chip.GPU)
    print("### original")
    print(df.collect())
    print("### non-literal from_base")
    print(spark.sql("SELECT CONV(a, b, 16) FROM TBL").collect())
    print("### 10 -> 16")
    print(df.select(pyspark.sql.functions.conv('a', 10, 16)).collect())
    print("### 16 -> 10")
    print(df.select(pyspark.sql.functions.conv('a', 16, 10)).collect())

### GPU

In [10]:
test_case(chip = Chip.GPU)

##### TEST chip=<Chip.GPU: 2>

### original
23/07/31 04:55:01 WARN GpuOverrides: 
! <RDDScanExec> cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.execution.RDDScanExec
  @Expression <AttributeReference> a#0 could run on GPU
  @Expression <AttributeReference> b#1 could run on GPU

[Row(a='gera', b=2), Row(a='231', b=10), Row(a='0xDEADBEEF', b=16)]
### non-literal from_base
23/07/31 04:55:01 WARN GpuOverrides: 
!Exec <ProjectExec> cannot run on GPU because not all expressions can be replaced
  @Expression <Alias> conv(a#0, b#1, 16) AS conv(a, b, 16)#13 could run on GPU
    !Expression <Conv> conv(a#0, b#1, 16) cannot run on GPU because only liter from_base and to_base are supported
      @Expression <AttributeReference> a#0 could run on GPU
      @Expression <AttributeReference> b#1 could run on GPU
      @Expression <Literal> 16 could run on GPU
  ! <RDDScanExec> cannot run on GPU because GPU does not currently support the operator class 

Py4JJavaError: An error occurred while calling o58.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (localhost executor driver): java.lang.UnsupportedOperationException
	at org.apache.spark.sql.rapids.GpuConv.doColumnar(stringFunctions.scala:2290)
	at com.nvidia.spark.rapids.GpuTernaryExpression.$anonfun$columnarEval$8(GpuExpressions.scala:413)
	at com.nvidia.spark.rapids.Arm$.withResourceIfAllowed(Arm.scala:74)
	at com.nvidia.spark.rapids.GpuTernaryExpression.$anonfun$columnarEval$7(GpuExpressions.scala:398)
	at com.nvidia.spark.rapids.Arm$.withResourceIfAllowed(Arm.scala:74)
	at com.nvidia.spark.rapids.GpuTernaryExpression.$anonfun$columnarEval$6(GpuExpressions.scala:397)
	at com.nvidia.spark.rapids.Arm$.withResourceIfAllowed(Arm.scala:74)
	at com.nvidia.spark.rapids.GpuTernaryExpression.columnarEval(GpuExpressions.scala:396)
	at com.nvidia.spark.rapids.Gpu

### CPU

In [ ]:
test_case(chip = Chip.CPU)